In [103]:
from pptx import Presentation
from pptx.util import Inches
import pandas as pd
import os
from pptx.util import Pt

In [104]:
csv_file = "iceland.csv"
df = pd.read_csv(csv_file,encoding="GBK")

In [105]:

### Text and images format setting.
left_position = [0.5,6,0.5,6]
top_position = [5,5,2,2]
text_top_offset = 0.4
text_left_offset = 0.0


prs = Presentation()

detail = pd.read_csv("media/detail.csv")
detail = detail.set_index("name")
for idx, row in df.iterrows():
    layout = row.layout
    title_text = row.title
    sub_text = str(row.subtitle)

    slide_layout = prs.slide_layouts[layout]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]

    title.text = title_text
    subtitle.text = sub_text
    
    #Add images:
    if row.type == 'img':
        src = "media/i"+row.folder+ "/"
        if not os.path.exists(src):
            os.makedirs(src)
        file_src = os.listdir(src)

        # Read every images in the folder.
        for idx, img_file in enumerate(file_src):
            if not img_file.endswith(".csv"):
                top = Inches(top_position[idx])
                left = Inches(left_position[idx])
                width = 3
#                 if detail.loc[src+img_file,'verti']:
                pic = slide.shapes.add_picture(src+img_file, left, top, width=Inches(width))
                height = pic.height.inches

                if detail.loc[src+img_file,'verti']:
                    pic.rotation = 90
                    text_top_offset = text_top_offset + (width-height)/2
                    text_left_offset = text_left_offset + (width-height)/2
                    print()
                # Add text to the picture
                txBox = slide.shapes.add_textbox(Inches(left_position[idx]+text_left_offset),
                                                 Inches(top_position[idx]-text_top_offset), Inches(1), Inches(1))
                text_frame = txBox.text_frame
                p = text_frame.paragraphs[0]
                run = p.add_run()
                run.text = str(detail.loc[src+img_file,'text'])
                font = run.font
                font.size = Pt(16)
                    
                    
    #insert video into the slide.
    if row.type == 'video':
        src = "media/v"+row.folder+ "/"
        if not os.path.exists(src):
            os.makedirs(src)
        file_src = os.listdir(src)
        top = Inches(2)
        left = Inches(3)
        width = 6
        height = width * detail.loc[src+vid, 'ratio' ]
        
        for vid in file_src:
            if (not vid.endswith(".csv")) and (not vid.endswith(".jpg")):
            # Ev
                movie = slide.shapes.add_movie(src+vid, left, top, Inches(width), Inches(height),
                                               poster_frame_image = src+"cover.jpg", mime_type='video/mov')
    
prs.save('test.pptx')

In [106]:
subtitle
# movie.top.inches